<a href="https://colab.research.google.com/github/Sisasenkosi/Diagnosis_Pneumonia_And_Lung-Opacity/blob/main/MedicalDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**THE IMPORTS**

In [ ]:
from keras.datasets import mnist
from keras.models import Model, load_model
from keras.utils import image_dataset_from_directory
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dense
from keras.layers import Input, Flatten, Rescaling

Mounting the drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


Copying the dataset into smaller dataset in new directories

In [ ]:
import os, shutil
import pathlib


original_path = pathlib.Path('/gdrive/MyDrive/DataSet') 
new_path = pathlib.Path('/gdrive/MyDrive/Colab Notebooks/Medical_smallerData')
def make_subset(subset_name, start_index, end_index):
    for category in ['Lung_Opacity','Normal','Viral Pneumonia']:
        folder = f'{new_path}/{subset_name}/{category}'
        os.makedirs(folder)
        

        filenames = [f'{category}-{i}.png' for i in range(start_index, end_index)]
        for filename in filenames:
            shutil.copyfile(src=f'{original_path}/{category}/{filename}', dst=f'{folder}/{filename}')

make_subset("train", start_index=1, end_index=1000)
make_subset("validation", start_index=1000, end_index=1200)
make_subset("test", start_index=1200, end_index=1340)

Setting the dataset paths in directories

In [ ]:
import pathlib
train_dataset = image_dataset_from_directory(
    directory= pathlib.Path ('/gdrive/MyDrive/Colab Notebooks/Medical_smallerData/train'),
    image_size=(180, 180),
    batch_size=32
)

val_dataset = image_dataset_from_directory(
    directory= pathlib.Path('/gdrive/MyDrive/Colab Notebooks/Medical_smallerData/validation'),
    image_size=(180, 180),
    batch_size=32
)

test_dataset = image_dataset_from_directory(
    directory= pathlib.Path ('/gdrive/MyDrive/Colab Notebooks/Medical_smallerData/test'),
    image_size=(180, 180),
    batch_size=32
)

Found 2997 files belonging to 3 classes.
Found 600 files belonging to 3 classes.
Found 420 files belonging to 3 classes.


Shape of training data

In [ ]:
for data, labels in train_dataset:
    print(data.shape, labels.shape)

(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3) (32,)
(32, 180, 180, 3

Defining the model

In [ ]:
inputs = Input(shape=(180,180,3))
x = Rescaling(scale=1/255)(inputs)
x = Conv2D(filters= 32, kernel_size=3, activation= 'relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters= 32, kernel_size=3, activation= 'relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(pool_size=2)(x)
#-------------------------------------
x = Flatten()(x)
x = Dense(units=10, activation='relu')(x)
outputs = Dense(units=3, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='rmsprop', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 32)       0         
 2D)                                                         

Defining the callbacks and training

In [ ]:
callbacks = [
    ModelCheckpoint(
        filepath=pathlib.Path('/gdrive/MyDrive/Colab Notebooks/best_model/best_model.hdf5'),
        save_best_only=True,
    ),
    EarlyStopping(
        patience=2
    )
]

history = model.fit(train_dataset, batch_size= 64,  epochs=6, callbacks=callbacks, validation_data=val_dataset)

Epoch 1/6
94/94 [==============================] - 147s 2s/step - loss: 0.3576 - accuracy: 0.8846 - val_loss: 0.3426 - val_accuracy: 0.8700
Epoch 2/6
94/94 [==============================] - 149s 2s/step - loss: 0.3296 - accuracy: 0.8986 - val_loss: 0.2213 - val_accuracy: 0.9283
Epoch 3/6
94/94 [==============================] - 148s 2s/step - loss: 0.2926 - accuracy: 0.9086 - val_loss: 0.2049 - val_accuracy: 0.9383
Epoch 4/6
94/94 [==============================] - 146s 2s/step - loss: 0.2698 - accuracy: 0.9129 - val_loss: 0.2713 - val_accuracy: 0.9050
Epoch 5/6
94/94 [==============================] - 146s 2s/step - loss: 0.2561 - accuracy: 0.9199 - val_loss: 0.1844 - val_accuracy: 0.9450
Epoch 6/6
94/94 [==============================] - 147s 2s/step - loss: 0.2415 - accuracy: 0.9246 - val_loss: 0.2172 - val_accuracy: 0.9367


Evaluating the model

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

14/14 [==============================] - 6s 385ms/step - loss: 0.2215 - accuracy: 0.9333


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label = 'Training loss')
plt.plot(history.history['val_loss'], label = 'Validation loss')
plt.legend()